<script type="text/javascript"  src="http://cdn.mathjax.org/mathjax/latest/MathJax.js?config=TeX-AMS-MML_HTMLorMML"></script>
# Multi-scale Convolutional Neural Network for Lung Nodule Classification

## 0. Abstract

우리는 흉부CT영상을 사용해 폐결절 진단 분류 문제를 다루는 법을 보여줄 것이다. 영역 분석(Regional Analysis)을 통한 결절 Segmentation에 의존한 전통적인 선행 연구들과는 다르게, 우리는 결절의 형태에 대한 어떤 사전정의도 없이 Nodule Patch를 직접 모델링하는 더 도전적인 문제에 도전했다. 우리는 인접하게 쌓인 층들로부터 구별되는 특징들을 추출해 결절의 이질성을 잡아내기 위해 **MCNN(Multi-scale CNN)**이라는 계층적 학습 구조를 제안한다. 특히 결절의 특징을 충분히 보장하기 위해 우리의 프레임워크는 Multi-scale Nodule Patch를 동시에 활용한다. 이것은 각각의 Scale의 마지막 층 활성함수들의 반응을 합쳐 Class-specific한 특징군을 학습하게 된다. 평가를 하기 위해 **LICD-IDRI라는 CT 데이터셋**을 사용했고 이것은 폐결절 영상과 주석이 포함되어 있다. 실험 결과는 결절 Segmentation 없이 악성 및 양성 결절을 분류하는 우리의 방법이 효과적임을 보여준다.

---
## 1. Introduction

### 과거의 연구들과 한계점

폐암은 장기적으로 낮은 생존률을 자랑하는 악명높은 병이다. 흉부CT를 사용한 폐결절 분석방법은 결절의 특징을 완벽히 잡아내는 강력한 도구였기 때문에 초기 암 진단의 중점적인 방법이었다. CT를 이용한 결절의 악성/양성 여부 분류는 방사선 전문의들이 결절의 진행정도를 평가하고 환자 개인마다의 치료 계획을 세울 수 있게 하기 때문에 매우 중요하다. Parametric Texture Feature Extraction같은 여러 접근법이 제안되었지만, 제대로 된 분석을 위해서는 알맞은 파라미터를 찾아야 한다는 점에서 문제가 있다. 즉, 이미지에 의한 질병 진단을 향상시키기 위해 임상데이터에서 의미있는 특징을 정의하고 추출해야 한다는 기술적 쟁점이 남아있다. 게다가 이전 연구들은 대부분 결절의 형태에 집중했다. 이것은 결절을 정확히 묘사할 수 없다. 예를 들어 결절 경계를 정의하는 일은 모호하고 주관적이기에, 방사선 전문의들의 의견차는 정확한 결절 묘사를 힘들게 한다. 이런 문제들로 미루어 봤을 때, 다음 두 가지 질문들이 떠오른다.

+ 서로 다른 진단 그룹을 나타내기 위해 heterogeneous한 결절 데이터로부터 discriminative한 feature들을 학습시키려면 어떻게 해야 하는가?
+ segmentation 과정을 생략해 segmented 결절이 아닌 원본 결절부위로부터 의미있는 feature들을 추출해낼 수 있는 일반적인 프레임워크를 어떻게 디자인해야 하는가?


### 우리의 연구

이 연구에서는 흉부CT에 기반한 폐결절 진단 분류 문제를 다룬다. segmentation이나 textural feature description 기반의 이전 방식과는 다르게 우리는 feature를 뽑아내기 위해 CNN을 활용한다. 추출된 feature들은 곧장 최신 분류기(SVM, Random Forest 등)에 넣을 수 있다. 우리의 방식은 오직 결절 데이터만을 이용해 86.84%의 분류 성능을 보여준다. 우리는 또한 이 방식이 노이즈에 강하다는 것을 발견했다. 분류 성능은 서로 다른 정도의 노이즈가 있는 input에 대해 꽤 안정적이었다.


### Contributions

+ MCNN은 segmented된 결절이 아닌 여러 층으로 구성된 **원본 결절 데이터**을 사용해 원본 데이터가 폐결절 진단에 유용하다는 증거를 제시한다.
+ MCNN은 **자동적으로 특징을 추출**함으로써, Nodule Texture, Shape Compactness, Nodule Sphericity 등의 수작업이 동반되는 feature 추출 작업을 제거한다.
+ MCNN은 **높은 노이즈에도 강한 모습**을 보여준다.


### Related Work

이미지 기반의 폐결절 분석은 일반적으로 nodule segmentation, feature extraction, 그리고 labeling nodule categories 등의 작업이 필요하다. 아래 연구들은 특징추출에 segmentation 작업이 선행된다. 하지만 자동 segmentation은 초기화 과정 등에 의해 분류 성능에 영향을 미칠 수 있다. 결과적으로 잘못된 특징을 추출해 전체 성능에 악영향을 미칠 수 있다.

+ Ted et al. : segmentation -> extracting texture features -> 선형 분류기로 학습
+ El-Baz et al. : 악성 폐결절 진단에 shape analysis 사용
+ Fangfang et al. : 양성 결절 진단에 3D texture feature 분석방법 사용


HOG(Histogram of Oriented Gradients)와 LBP(Local Binary Patterns)는 메디컬 이미지 분석 평가에 널리 사용된다. 하지만 이 방식은 hyperparameter마다 다른 성능을 보여주고 결과를 미리 예측할 수 없다. 따라서 매개변수값을 일일히 변경해가며 반복 테스트를 해야 한다.

우리의 연구는 개념적으로는 Massive Training Artificial Neural Network와 흡사하다. 하지만 이 개념은 의사결정을 위해 2D 가우시안 분포를 포함시켜 학습된 local pixel들로부터 image-to-image mapping을 수행하는 분류기이다. 반면 우리의 접근법은 분포를 사용하지 않고 여러 convolutional operation과 max-pooling을 통해 원본 결절 이미지로부터 특징 추출을 하는데 중점을 둔다. 또한 feature dimension을 줄여 연산이 효과적이다.  

---
## 2. Learning Multi-scale CNN

우리의 목표는 주어진 폐결절 CT 이미지를 사용해 MCNN 모델로 discriminative feature들을 찾아내는 것이다. 문제는 데이터에 건강한 조직과 결절이 한데 묶여있기 때문에 이미지 공간이 서로 굉장히 다르다는 점이다. 그래서 우리는 3개의 CNN을 병렬로 연결해 각기 다른 크기의 결절들로부터 multi-scale feature를 추출하게 만들었다.

### (1) CNN 구조

우리의 CNN은 두 convolutional layer로 구성되어 있고, 각각 뒤에 max-pooling layer가 붙어있으며, 마지막 층은 fully-connected layer이다. (그림참조)

### (2) Multi-scale Nodule Representation

+ **데이터셋** : ***LIDC-IDRI*** 데이터셋에는 3mm부터 30mm까지 크기가 다양한 결절들이 존재한다는 점에서 착안해 multi-scale 전략을 채택
+ **구조** : 세 CNN이 병렬 연결되어있는 형태이며, 각각을 \\({CNN}_{0}\\), \\({CNN}_{1}\\), \\({CNN}_{2}\\)라고 한다.
+ **Parameter** : 학습과정을 단순화시키기 위해 모든 CNN은 같은 parameter를 공유한다.
+ **Input** : 각 CNN에 한 가지 결절에 대한 다른 세 가지 scale의 input이 들어간다.  
+ **Output** : 세 CNN output을 종합한 것이며 feature vector의 reduction 없이 최종 분류기로 들어간다.



---
## 3. Experiments

### (1) Dataset and Setup

+ **악성/양성의 기준 설정** : LIDC-IDRI 데이터셋에는 1010명의 환자의 폐CT 스캔과 주석이 달려있다. 결절은 숙련된 전문의에 의해 1~5로 평가되며, 악성 종양의 정도가 클수록 숫자도 커진다. 또한 '레퍼런스8'의 방식대로 각 결절들에 대한 평균 악성 정도를 평가해, 평균점수가 3 미만이면 양성, 3 초과면 악성으로 판단했다. 애매한 결절이나 평균점수 3의 결절은 제거했다. 결과적으로 880개의 양성 결절과 495개의 악성 결절로 나뉘었다. ***(※ Binary Classification을 하기 위해 기존의 5단계 분류를 악성/양성으로 나눈 것)***
+ **해상도 설정** : 이미지의 해상도가 다양했기 때문에 세 scale 모두 0.5mm/pixel로 고정해 해상도의 영향력을 제거했다. 이 연구에서는 추가로 http://www.via.cornell.edu/lidc/ 에서 얻어온 결절의 중심좌표를 활용했는데, 표시된 결절 중심좌표를 기준으로 각 결절조각들을 세 종류의 크기로 잘라냈다. 조각들은 32x32x32, 64x64x64, 96x96x96 픽셀이며, 이후 CNN에 넣기 위해 32x32x32로 리사이징했다.

### (2) Implementation Details

+ Validation Method : 5-fold cross validation 
+ Dataset Settings
  + 1100 training set (양성 704 + 악성 396), 275 testing set (양성 176 + 악성 99)
  + 세 축을 2픽셀씩 늘이고 줄이며 한 조각당 6개의 이미지를 파생시켜 데이터셋을 늘리는 방식 채택 (레퍼런스 16)
  + 악성결절 데이터셋이 부족하므로, 악성결절은 위 방식으로 파생시키고 양성결절은 절반만 취해 균형을 맞춤
  + 결과적으로 5588개의 균형잡힌 데이터셋을 마련함
+ CNN은 서로 parameter를 공유하고 각각 5588개의 데이터셋을 사용하므로 총 16764개의 이미지가 사용됨
+ testing set은 각 validation round마다 275개씩 고정되게 사용됨

+ 각 층마다 서로 다른 숫자의 convolutional kernel을 사용 (\\({n}_1 {= (50, 100)}, {n}_2 {= (50, 100)}, {n}_3 {= (20, 50)}\\))
+ Companion Objective : 0.001 (레퍼런스 12에서 최적화된 값)
+ Convolutional Kernel Size : 5x5xK (K는 input feature map의 세번째 차원값)
+ Pooling Size : 2x2 Window
+ L2 norm weight decay : overfitting 방지를 위해
+ Classifiers : 두 가지 분류기 사용 (RBF kernel이 포함된 SVM, RF), hyperparameters는 노가다로 최적화
+ 결과비교 : 두 가지 비교기 사용 (HOG, LBP descriptor)
  + HOG : 
  + LBP : 
+ 기타 : NVIDIA Tesla K40 GPU, CAFFE, scikit-learn의 SVM과 RF 사용, scikit-image의 HOG와 LBP 사용

### (3) Binary Nodule Classification Results

figure 및 table 참조

### (4) Robustness to Noise Corruption

데이터셋에 노이즈를 적용하면 어떤 결과가 나올까? Gaussian Noise를 각기 다른 파라미터로 적용해 보았는데, 놀랍게도 \\(\sigma { = 2.0} \\)에서 83.56%에서 82.27%로밖에 낮아지지 않으며 정확도 손실이 크지 않았다. 아마도 max-pooling layer에서 걸러지기 때문인 것 같다.

### (5) Discussion

이상 MCNN 프레임워크의 결절 진단 분류기의 유망한 결과를 볼 수 있었다. CNN은 이미지의 특성을 강력하고 정확하게 파악하기 위한 업무에 쉽게 적용 가능한 모습을 보인다. 비록 성능 향상을 위해 결절의 적절한 크기(scale)를 조정할 필요가 있겠지만, 우리 연구에서는 단 하나의 scale을 사용하더라도 각 CNN들이 86.12%, 83.88%, 79.00%의 정확도를 기록했다. 하지만 multi-scale 전략을 사용함으로서 이미지 scale에 대한 세심한 고려의 필요성을 제거했다. 또한 isotropic한 커널을 사용하는 대신 커널의 3번째 차원을 input feature map에 일치시켰다. 이것은 CNN의 중간층에 일반적으로 사용되는 방식인데, 덕분에 CAFFE의 내부를 건드리지 않고 바로 사용할 수 있었다.

---
## 4. Conclusion

이 논문에서는 폐결절 진단 분류기를 구성하기 위해 MCNN을 도입했다. 우리는 학습을 통해 얻은 대표적인 feature들이 서로 다른 결절을 구분해낼 수 있다는 것을 시연했다. MCNN이 노이즈에 강하다는 사실 또한 메디컬 이미지 분석 업계에서 희망적인 부분이다. 추가 실험으로 86.84%의 정확도를 달성하며 주석에 의한 진단의 정확도를 뛰어넘었다. 앞으로는 이것을 대규모 데이터에 적용해 볼 것이고, 각 이미지의 가장 적절한 크기를 알아볼 것이다.